Na początek ładuje rmagic aby móc korzystać z R w colab

In [ ]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


Wczytanie zbioru danych i wyświetlenie jego właściwości. Dowiaduje się jakie są atrybuty, ile jest obiektów, jakie są typy atrybutów oraz jaki jest atrybut decyzyjny.

In [ ]:
%%R
getwd()
data <- read.table(file = '/content/drive/MyDrive/us/SIwB/CzyszczenieDanych/TitanicMess.tsv', sep = '\t', header = TRUE, na.strings=c("","NA")) # wczytanie pliku
head(data)
summary(data) # jakie są w zbiorze atrybuty, ile jest obiektów, typy atrybutów, atrybut decyzyjny

  PassengerId        Survived          Pclass          Name          
 Min.   :   1.0   Min.   :0.0000   Min.   :1.000   Length:892        
 1st Qu.: 223.8   1st Qu.:0.0000   1st Qu.:2.000   Class :character  
 Median : 444.5   Median :0.0000   Median :3.000   Mode  :character  
 Mean   : 445.8   Mean   :0.3868   Mean   :2.307                     
 3rd Qu.: 668.2   3rd Qu.:1.0000   3rd Qu.:3.000                     
 Max.   :1000.0   Max.   :1.0000   Max.   :3.000                     
     Sex                Age                SibSp            Parch       
 Length:892         Length:892         Min.   :0.0000   Min.   :0.0000  
 Class :character   Class :character   1st Qu.:0.0000   1st Qu.:0.0000  
 Mode  :character   Mode  :character   Median :0.0000   Median :0.0000  
                                       Mean   :0.5258   Mean   :0.3711  
                                       3rd Qu.:1.0000   3rd Qu.:0.0000  
                                       Max.   :8.0000   Max.   :5.0000  

Pobranie nazw kolumn i wyświetlenie liczby wartości brakujących w poszczególnych kolumnach:

In [ ]:
%%R
library(tidyverse)
columns <- colnames(data)
glimpse(data)
print(columns)
for(i in columns) {
    cat("Liczba brakujących wartości w atrybucie", i, ":", sum(is.na(data[[i]])), "\n",sep=" ")
}

Rows: 892
Columns: 13
$ PassengerId <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 11, 15, 16, 17,…
$ Survived    <int> 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1…
$ Pclass      <int> 3, 1, 3, 1, 3, 3, 1, 3, 3, 2, 3, 1, 3, 3, 3, 2, 3, 2, 3, 3…
$ Name        <chr> "Braund, Mr. Owen Harris", "Cumings, Mrs. John Bradley (Fl…
$ Sex         <chr> "male", "female", "female", "female", "male", "male", "mal…
$ Age         <chr> "22", "38", "26", "35", "35", NA, "54", "2", "27", "14", "…
$ SibSp       <int> 1, 1, 0, 1, 0, 0, 0, 3, 0, 1, 1, 0, 0, 1, 0, 0, 4, 0, 1, 0…
$ Parch       <int> 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0…
$ Ticket      <chr> "A/5 21171", "PC 17599", "STON/O2. 3101282", "113803", "37…
$ Fare        <chr> "7,25", "71,2833", "7,925", "53,1", "8,05", "8,4583", "51,…
$ Cabin       <chr> NA, "C85", NA, "C123", NA, NA, "E46", NA, NA, NA, "G6", "C…
$ Embarked    <chr> "S", "C", "S", "S", "S", "Q", "S", "S", "S", "C", "S", "S"…
$ ship        <chr

Teraz w celu zwizualizowania zbioru wyświetlam wykresy dla poszczególnych atrybutów. Z wykresów odczytać można że pewne id powtarzają się, atrybuty "Sex" oraz "Age" oraz "Embarked" mają nieprawidłowe wartości. Kolumna "Ship" jest nadmiarowa, gdyż zawiera tylko jedną wartość.

In [ ]:
%%R
for(i in columns) {
    barplot(table(data[[i]]), main = i)
}

Z wykresów wyniknęły następujące błędy


*   elementy o powtarzającym się ID - należy usunąć powielone
*   Atrybut 'Sex' posiada nieprawidłowe wartości
*   Atrybut 'Age' posiada nieprawidłowe wartości
*   Atrybut 'Fare' posiada nieprawidłowe wartości
*   Atrybut 'Embarked' posiada nieprawidłowe wartości
*   Atrybut 'Ship' jest nadmiarowy - kolumnę można usunąć gdyż zawiera tylko jedną wartość

Teraz należy odszukać niepoprawne wartości dla atrybutów Age oraz Fare. W tym celu przeszukuję zbiór w pętli: 

In [ ]:
%%R
dat1 <- data
wrong_age_ids <- c()
wrong_fare_ids <- c()
for(i in 1:nrow(dat1))  {
  if(!is.na(dat1[i, 6])) { # pomijam sprawdzanie dla wartości brakujących
    age <- as.double(gsub(",",".",dat1[i, 6])) # konwersja na liczbę przy czym trzeba zamienić przecinek na kropkę
	  if(age > 120 | age < 0) { # szukam wartości błędnych, 
	    wrong_age_ids <- c(wrong_age_ids, dat1[i, 1]) # zbieram id-ki z błędnymi wartościami wieku
		 	cat("Nieprawidłowa wartość Age: ", age,"\n",sep=" ")
	  }   
  }
  if(!is.na(dat1[i, 10])) { # pomijam sprawdzanie dla wartości brakujących
    fare <- as.double(gsub(",",".",dat1[i, 10]))
	  if(is.na(fare) | fare < 0) { # szukam wartości błędnych - nie double i ujemne
	    wrong_fare_ids <- c(wrong_fare_ids, dat1[i, 1]) # zbieram id-ki z błędnymi wartościami fare
		  cat("Nieprawidłowa wartość Fare: ", dat1[i, 10],"\n",sep=" ")
	  }   
  }
}
cat("wiersze o błędnym Age do usunięcia: ", wrong_age_ids,"\n",sep=" ")
cat("wiersze o błędnym Fare do usunięcia: ", wrong_fare_ids,"\n",sep=" ")


Nieprawidłowa wartość Age:  4435 
Nieprawidłowa wartość Fare:  -90 
Nieprawidłowa wartość Fare:  15,9a 
Nieprawidłowa wartość Age:  -3 
Nieprawidłowa wartość Age:  -12 
Nieprawidłowa wartość Age:  250 
wiersze o błędnym Age do usunięcia:  224 508 523 667 
wiersze o błędnym Fare do usunięcia:  246 490 


Teraz oczyszczam zbior danych z wartości nieprawidłowych

In [ ]:
%%R
dat2 <- data
dat2 <- dat2[!duplicated(dat2$PassengerId), ] # usunięcie zduplikowanych wierszy na podstawie ID
dat2 <- dat2[dat2$Sex %in% c("male", "female"),] # usunięcie wierszy z błędem w płci.
dat2 <- dat2[dat2$Embarked %in% c("S", "Q", "C") | is.na(dat2$Embarked),] # usunięcie wierszy z błędem w embarked.
dat2 <- dat2[!(dat2$PassengerId %in% wrong_age_ids),] # usunięcie wierszy z błednym wiekiem
dat2 <- dat2[!(dat2$PassengerId %in% wrong_fare_ids),] # usunięcie wierszy z fare innym niż double i ujemne
dat2 <- dat2[,names(dat2) != "ship"] # usunięcie kolumny ship - bo zawiera tylko jedną wartość - jest nadmiarowa
glimpse(dat2)

Rows: 875
Columns: 12
$ PassengerId <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 19,…
$ Survived    <int> 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1…
$ Pclass      <int> 3, 1, 3, 1, 3, 3, 1, 3, 3, 2, 3, 1, 3, 3, 2, 3, 3, 3, 2, 3…
$ Name        <chr> "Braund, Mr. Owen Harris", "Cumings, Mrs. John Bradley (Fl…
$ Sex         <chr> "male", "female", "female", "female", "male", "male", "mal…
$ Age         <chr> "22", "38", "26", "35", "35", NA, "54", "2", "27", "14", "…
$ SibSp       <int> 1, 1, 0, 1, 0, 0, 0, 3, 0, 1, 1, 0, 0, 0, 0, 4, 1, 0, 0, 0…
$ Parch       <int> 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0…
$ Ticket      <chr> "A/5 21171", "PC 17599", "STON/O2. 3101282", "113803", "37…
$ Fare        <chr> "7,25", "71,2833", "7,925", "53,1", "8,05", "8,4583", "51,…
$ Cabin       <chr> NA, "C85", NA, "C123", NA, NA, "E46", NA, NA, NA, "G6", "C…
$ Embarked    <chr> "S", "C", "S", "S", "S", "Q", "S", "S", "S", "C", "S", "S"…


Następnie zastępuje brakujące wartości (NA) atrybutów Age oraz Embarked. Ponadto zmieniam format Age oraz Fare na double gdyż są to liczby zmiennoprzecinkowe. Atrybut Cabin posiada wiele brakujących wartości, ale nie da się ich zastąpić a usunięcie tych danych spowodowało by znaczną redukcje rozmiaru zbioru.



In [ ]:

%%R
dat3 <- dat2
print(dat3[is.na(dat3$Embarked),])
dat3$Embarked[which(is.na(dat3$Embarked))] <-"s" # zastąpienie Na przez "s", bo najczęściej występuje
print(dat3[is.na(dat3$Embarked),])
dat3$Age[which(is.na(dat3$Age))] <- mean(as.double(gsub(",",".",dat3$Age)), na.rm=TRUE) # zastąpienie Age przez mean z całego zbioru
dat3$Age <- sapply(dat3$Age, as.double) # zmiana formatu na double
dat3$Fare <- sapply(gsub(",",".",dat3$Fare), as.double) # zmiana formatu na double
write.table(dat3, file = '/content/drive/MyDrive/us/SIwB/CzyszczenieDanych/TitanicCleaned.tsv', quote=FALSE, sep='\t')
# write.csv(dat3, file = '/content/drive/MyDrive/us/SIwB/CzyszczenieDanych/TitanicCleaned.csv')
glimpse(dat3)

    PassengerId Survived Pclass                                      Name
62           62        1      1                       Icard, Miss. Amelie
830         830        1      1 Stone, Mrs. George Nelson (Martha Evelyn)
       Sex Age SibSp Parch Ticket Fare Cabin Embarked
62  female  38     0     0 113572   80   B28     <NA>
830 female  62     0     0 113572   80   B28     <NA>
 [1] PassengerId Survived    Pclass      Name        Sex         Age        
 [7] SibSp       Parch       Ticket      Fare        Cabin       Embarked   
<0 rows> (or 0-length row.names)
Rows: 875
Columns: 12
$ PassengerId <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 19,…
$ Survived    <int> 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1…
$ Pclass      <int> 3, 1, 3, 1, 3, 3, 1, 3, 3, 2, 3, 1, 3, 3, 2, 3, 3, 3, 2, 3…
$ Name        <chr> "Braund, Mr. Owen Harris", "Cumings, Mrs. John Bradley (Fl…
$ Sex         <chr> "male", "female", "female", "female", "male", "male", "mal…
$ Age  